In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import integrate
from importlib import reload
import datetime as dt
sys.path.append(r'D:\07. Software\jupyter')
from utils_gfat import lidar
from utils_gfat import lidar_elastic_retrieval
import lidar_processing.lidar_processing.helper_functions as helper_functions
import lidarQA
import pdb

In [ ]:
def klett_loop_sims(rcs, beta_mol, lr_aer =45, ymin = 8000, ymax = 8500):
    cf = (rcs.sel(range=slice(ymin,ymax)).mean(axis=0)/beta_mol.sel(range=slice(ymin,ymax)).mean(axis=0)).values        
    beta = np.zeros(len(rcs)) 
    integral_up = np.zeros(len(rcs)) 
    integral_down = np.zeros(len(rcs)) 
    beta_mol = beta_mol.values
    resol = np.median(np.diff(rcs['range']))
    rcs = rcs.values    
    for idx in np.arange(len(rcs)-1):
        integral_up[idx] = integrate.simps(2*(lr_mol-lr_aer)*beta_mol[:idx+1], dx=resol)
        integral_down[idx] = integrate.simps(lr_aer*rcs[0:idx+1]*np.exp(integral_up[idx+1]), dx=resol)
        beta[idx] = (rcs[idx]*np.exp(integral_up[idx]))/(cf - 2* integral_down[idx]) - beta_mol[idx] 
    return beta, integral_up, integral_down

def klett(rcs, beta_mol, lr_aer =45, ymin = 8000, ymax = 8500):
    cf = (rcs.sel(range=slice(ymin,ymax)).mean(axis=0)/beta_mol.sel(range=slice(ymin,ymax)).mean(axis=0)).values
    print(cf)    
    integral_up =integrate.cumtrapz((lr_mol-lr_aer)*beta_mol, x=height,initial=0)
    integral_down = integrate.cumtrapz(lr_aer*rcs*np.exp(2*integral_up), x=height,initial=0)        
    beta = (rcs*np.exp(2*integral_up))/(cf - 2* integral_down) - beta_mol    
    return beta, integral_up, integral_down

def quasi_klett_jaba(beta_att, beta_mol, resolution= 7.5, lr_aer =45, ymin = 8000, ymax = 8500):
    lr_mol = 8*np.pi/3
    
    #Quasi Step 0
    alpha_mol = beta_mol*lr_mol
    aod_mol = integrate.cumtrapz(alpha_mol, dx=resolution, initial=0)
    Tmol = np.exp(-aod_mol)
    Taer = 1
    Ttotal0 = Tmol*Taer
    qbeta0 = beta_att/Ttotal0**2 - beta_mol

    #Quasi Step 1
    alpha_aer1 = qbeta0*lr_aer
    aod_aer1 = integrate.cumtrapz(alpha_aer1, dx=resolution, initial=0)
    Ttotal1 = np.exp(-(aod_mol + aod_aer1))
    qbeta1 = beta_att/Ttotal1**2 - beta_mol
    return qbeta1 

In [ ]:
rs_fl = os.path.join(r'Y:\datos\MULHACEN\1a\2021\07\11', 'mhc_1a_Prs_20210711.nc')  
dc_fl = os.path.join(r'Y:\datos\MULHACEN\1a\2021\07\11', 'mhc_1a_Pdc_20210711_2128.nc')  
outputdir = r'C:\Users\Usuario\Downloads'

In [ ]:
wds = lidar.preprocessing(rs_fl, dc_fl, channels=[6])

In [ ]:
# Molecular Attenuated Backscatter
wavelength = 1064
# standard atmosphere profile:
heights = wds['range'].values
Tsa = np.ones(heights.size)*np.nan
Psa = np.ones(heights.size)*np.nan
for i, _height in enumerate(heights):
    sa = helper_functions.standard_atmosphere(_height)
    Psa[i] = sa[0]
    Tsa[i] = sa[1]
pressure_prf = Psa 
temperature_prf = Tsa 
beta_mol_att = lidarQA.molecular_attenuated_backscatter(wavelength, pressure_prf, temperature_prf, heights)
beta_mol = helper_functions.molecular_backscatter(wavelength, pressure_prf, temperature_prf)
atmospheric_profiles = xr.Dataset({'abetamol': (['range'], beta_mol_att), 'betamol': (['range'], beta_mol)}, coords={'range': heights})

In [ ]:
ini_date, end_date='20210711T073000', '20210711T083000'
ymin = 8000
ymax = 9000
lr_mol = 8*np.pi/3
lr_aer = 45
beta_mol = atmospheric_profiles['betamol'].sel(range=slice(0, ymax))
rcs = wds['corrected_rcs_06'].sel(time=slice(ini_date, end_date),range=slice(0, ymax)).mean(dim='time')
height = wds['range'].sel(range=slice(0, ymax))

#cumtrapz
beta_jaba, integral_up_jaba, integral_down_jaba = klett(rcs, beta_mol, lr_aer =45)
#Sims
beta_sims, integral_up_sims, integral_down_sims = klett_loop_sims(rcs, beta_mol, lr_aer =45)

#Bottom_up
betamol2rcs_ref = (beta_mol.sel(range=slice(ymin,ymax)).mean(axis=0)/rcs.sel(range=slice(ymin,ymax)).mean(axis=0)).values
beta_att = rcs*betamol2rcs_ref
fulloverlap_bin = lidar_elastic_retrieval.get_ref_height_index(heights, 0, 300)[0]
beta_att.load()[0:fulloverlap_bin] = beta_att[fulloverlap_bin].values
beta_bottom, info_bottom = lidar_elastic_retrieval.bottom_up_backscatter_aerosol(beta_att, height, beta_mol, lr_mol, lr_aer)


#Quasi Step 0
alpha_mol = beta_mol*lr_mol
aod_mol = integrate.cumtrapz(alpha_mol, x=height,initial=0)
Tmol = np.exp(-aod_mol)
Taer = 1
Ttotal0 = Tmol*Taer
qbeta0 = beta_att/Ttotal0**2 - beta_mol

#Quasi Step 1
alpha_aer1 = qbeta0*lr_aer
aod_aer1 = integrate.cumtrapz(alpha_aer1, x=height,initial=0)
Ttotal1 = np.exp(-(aod_mol + aod_aer1))
qbeta1 = beta_att/Ttotal1**2 - beta_mol

#Quasi Step 2
alpha_aer2 = qbeta1*lr_aer
aod_aer2 = integrate.cumtrapz(alpha_aer2, x=height,initial=0)
Ttotal2 = np.exp(-(aod_mol + aod_aer2))
qbeta2 = beta_att/Ttotal2**2 - beta_mol

#Quasi Step 3
alpha_aer3 = qbeta2*lr_aer
aod_aer3 = integrate.cumtrapz(alpha_aer3, x=height,initial=0)
Ttotal3 = np.exp(-(aod_mol + aod_aer3))
qbeta3 = beta_att/Ttotal3**2 - beta_mol


#Figure
fig, ax = plt.subplots(1,3, figsize=[15,10])
beta_jaba.plot(y='range', label='klett-JABA', c='k', lw=2,ax=ax[0]) 
beta_bottom.plot(y='range', label='bottom-up', c='b', lw=2,ax=ax[0]) 
ax[0].plot(beta_mol, height)
qbeta0.plot(y='range', label='0',ax=ax[0])
qbeta1.plot(y='range', label='1',ax=ax[0])
qbeta2.plot(y='range', label='2', ls='dashed',ax=ax[0])
qbeta3.plot(y='range', label='3',ax=ax[0])
# beta_klett.plot(y='range', label='Klett-ioannis', c='b', lw=1, ax=ax[0])

# ax[0].plot(beta_sims, height,label='sims', c='r', ls='dashed',lw=1)

# ax[1].plot((qbeta0-beta_jaba), height, label='0')
ax[1].plot(100*(qbeta1-beta_jaba)/beta_jaba, height, label='1')
ax[1].plot(100*(qbeta2-beta_jaba)/beta_jaba, height, label='2')
ax[1].plot(100*(qbeta3-beta_jaba)/beta_jaba, height, label='3')
ax[1].plot(100*(beta_bottom-beta_jaba)/beta_jaba, height, label='bottom',c='b')
ax[1].set_xlim(-20,20)
rcs.plot(y='range', ax=ax[2])
# beta_att.plot(y='range', ax=ax[2])

ax[0].legend(fontsize='medium')
ax[1].legend(fontsize='medium')
